In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [3]:
DELETE_TABLES=True
if DELETE_TABLES:
    try:
        cmd="""DROP TABLE `%s`"""%osm_intersections_table_name
        cursor.execute(cmd)
    except Exception as e:
        print(e)
        pass
    cmd="""DROP TABLE `%s`"""%osm_nodes_table_name
    cursor.execute(cmd)
2

2

In [4]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
    `osm_id` BIGINT NOT NULL,
    `contour_line_id` INT,
    `geometry` POINT NOT NULL,
    `elevation` FLOAT,

    FOREIGN KEY (contour_line_id) REFERENCES %s(id),
    PRIMARY KEY (osm_id)
    );
        """%(osm_nodes_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)



    
    cmd="""CREATE TABLE `%s`(
        `osm_begin` BIGINT NOT NULL,
        `osm_end` BIGINT NOT NULL,
        `osm_key` INT NOT NULL,
        `contour_line_id` INT NOT NULL,
        `edge_coordinate` FLOAT NOT NULL,
        `geometry` POINT NOT NULL,
        `elevation` FLOAT NOT NULL,

        FOREIGN KEY (osm_begin) REFERENCES %s(osm_id),
        FOREIGN KEY (osm_end) REFERENCES %s(osm_id),
        FOREIGN KEY (contour_line_id) REFERENCES %s(id),
        PRIMARY KEY (osm_begin,osm_end,osm_key,edge_coordinate)
        

          )
          ENGINE=InnoDB;
        """%(osm_intersections_table_name,osm_nodes_table_name,osm_nodes_table_name,contours_lines_table_name)
    cursor.execute(cmd)
    

2

2

In [5]:
t1=time.time()
G=build_graph(cursor)
max_depth=np.max([data['depth'] for _,data in G.nodes(data=True)])
t2=time.time()
print(t2-t1)

0.6394138336181641


In [6]:
t1=time.time()
place_name='Corse, France'
G_osm=ox.graph_from_place(place_name,network_type='all_private')
G_osm=to_multi_graph(G_osm)
osm_crs=ox.graph_to_gdfs(G_osm,edges=False).estimate_utm_crs()
G_osm=ox.project_graph(G_osm,to_crs=osm_crs)
add_missing_geometries(G_osm)
t2=time.time()
print(t2-t1)
len(G_osm.nodes()),len(G_osm.edges())

540.1164090633392


(83574, 102644)

In [7]:
t1=time.time()
osm_edges_df=ox.graph_to_gdfs(G_osm,nodes=False,edges=True)
osm_edges_df['edge']=osm_edges_df.index
osm_edges_df=osm_edges_df.loc[:,['edge','geometry']]
osm_edges_df['number_of_points']=osm_edges_df['geometry'].apply(lambda ls:len(ls.coords))
#osm_edges_df=osm_edges_df.sort_values(by='length')
osm_edges_df=osm_edges_df.set_index(np.arange(len(osm_edges_df)))
osm_edges_df['id_edge']=osm_edges_df.index
t2=time.time()
print(t2-t1)
osm_edges_df

2.900575637817383


,edge,geometry,number_of_points,id_edge
0,"(14468318, 15905423, 0)","LINESTRING (525561.716 4599894.093, 525588.264...",5,0
1,"(14468318, 510388434, 0)","LINESTRING (525561.716 4599894.093, 525576.103...",3,1
2,"(14468318, 5857486694, 0)","LINESTRING (525561.716 4599894.093, 525551.015...",3,2
3,"(510388434, 1434053246, 0)","LINESTRING (525589.280 4599947.781, 525609.009...",8,3
4,"(510388434, 1434053223, 0)","LINESTRING (525589.280 4599947.781, 525578.049...",8,4
...,...,...,...,...
102639,"(11304366184, 11304366185, 0)","LINESTRING (494097.672 4719579.054, 494140.238...",2,102639
102640,"(11304366184, 11304366187, 0)","LINESTRING (494097.672 4719579.054, 494085.250...",3,102640
102641,"(11304366220, 11304366221, 0)","LINESTRING (494208.672 4719716.551, 494192.415...",2,102641
102642,"(11304366220, 11304366222, 0)","LINESTRING (494208.672 4719716.551, 494237.383...",2,102642


In [8]:
t1=time.time()
intersection,depth=decreasing_depth_intersections(osm_edges_df,osm_crs,G,cursor,max_depth,depth_step=20,max_delta_time_total=float('inf'),max_delta_time_per_level=float('inf'))
t2=time.time()
print('total : %f'%(t2-t1),depth)

520<depth<=540 :0.591641 
500<depth<=520 :0.321242 
480<depth<=500 :0.397195 
460<depth<=480 :0.517075 
440<depth<=460 :0.713617 
420<depth<=440 :1.040093 
400<depth<=420 :1.465574 
380<depth<=400 :1.971526 
360<depth<=380 :2.536559 
340<depth<=360 :2.976114 
320<depth<=340 :3.496291 
300<depth<=320 :4.410798 
280<depth<=300 :5.412817 
260<depth<=280 :6.398170 
240<depth<=260 :7.579466 
220<depth<=240 :12.080304 
200<depth<=220 :14.664861 
180<depth<=200 :18.078570 
160<depth<=180 :22.152777 
140<depth<=160 :26.535479 
120<depth<=140 :31.580522 
100<depth<=120 :43.252684 
80<depth<=100 :62.257083 
60<depth<=80 :68.682474 
40<depth<=60 :70.869755 
20<depth<=40 :84.430694 
0<depth<=20 :133.674651 
total : 628.097715 1


In [9]:
t1=time.time()
pre_osm_nodes_to_contours={}
for edge,df in intersection.groupby('edge'):
    df=df.sort_values('edge_coordinate')
    u,v=edge[:2]
    if not u in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[u]={df.iloc[0]['id']}
    else:
        pre_osm_nodes_to_contours[u].add(df.iloc[0]['id'])
    if not v in pre_osm_nodes_to_contours:
        pre_osm_nodes_to_contours[v]={df.iloc[-1]['id']}
    else:
        pre_osm_nodes_to_contours[v].add(df.iloc[-1]['id'])
t2=time.time()
print(t2-t1)

87.76856279373169


In [10]:
t1=time.time()
G_osm.remove_edges_from(intersection['edge'].unique())
t2=time.time()
print(t2-t1)

0.1915590763092041


In [11]:
t1=time.time()
CC=list(nx.connected_components(nx.Graph(G_osm)))
pre_CC_to_contours={}
for k,cc in enumerate(CC):
    contours_ids=set()
    for osm_node in cc:
        if osm_node in pre_osm_nodes_to_contours:
            contours_ids=contours_ids.union(pre_osm_nodes_to_contours[osm_node])
    pre_CC_to_contours[k]=contours_ids

t2=time.time()
print(t2-t1)

0.6267440319061279


In [12]:
t1=time.time()
CC_to_contours={}
ambiguous_CC={}
for k,contour_ids in pre_CC_to_contours.items():
    if len(contour_ids)>1:
        id_1,id_2=list(contour_ids)[:2]
        if id_2 in G.neighbors(id_1):
            CC_to_contours[k]=id_1
        elif id_1 in G.neighbors(id_2):
            CC_to_contours[k]=id_2
        else:
            predecessors=list(G.predecessors(id_2))
            if len(predecessors)>0:
                CC_to_contours[k]=predecessors[0]
            else:
                print('component is outside with %is point'%len(list(CC[k])))

    else:
        ambiguous_CC[k]=list(contour_ids)[0]

t2=time.time()
print(t2-t1)

0.17174339294433594


In [13]:
contour_id_bunch_size=20

t1=time.time()
ambiguous_CC=pd.DataFrame([{'id_cc':k,'ambiguous_id_contour':v} for k,v in ambiguous_CC.items()])
ambiguous_contours_ids=ambiguous_CC['ambiguous_id_contour'].unique()
for i in range(0,len(ambiguous_contours_ids),contour_id_bunch_size):
    t3=time.time()
    ambiguous_contours_df=get_nodes_data(cursor,ambiguous_contours_ids[i:min(i+contour_id_bunch_size,len(ambiguous_contours_ids))])
    ambiguous_contours_df=ambiguous_contours_df.set_index('id')
    ambiguous_contours_df=ambiguous_contours_df.to_crs(osm_crs)
    ambiguous_contours_df['geometry']=ambiguous_contours_df['geometry'].apply(lambda ls:Polygon(ls))
    t4=time.time()
    for ambiguous_id_contour,row in ambiguous_contours_df.iterrows():
        polygon=row['geometry']
        for id_cc in ambiguous_CC[ambiguous_CC.ambiguous_id_contour==ambiguous_id_contour]['id_cc']:
            osm_node=random.choice(list(CC[id_cc]))
            datum=G_osm.nodes()[osm_node]
            pt=Point(datum['x'],datum['y'])
            if pt.within(polygon):
                CC_to_contours[id_cc]=ambiguous_id_contour
            else:
                predecessors=list(G.predecessors(ambiguous_id_contour))
                if len(predecessors)>0:
                    CC_to_contours[id_cc]=predecessors[0]
                else:
                    print('component is outside with %is point'%len(list(CC[id_CC])))
    t5=time.time()
    print('%i/%i : getting df : %f, check intersections : %f'%(i,len(ambiguous_contours_ids),t4-t3,t5-t4))

t2=time.time()
print(t2-t1)

0/2851 : getting df : 3.721984, check intersections : 124.260568
20/2851 : getting df : 1.447107, check intersections : 35.462979
40/2851 : getting df : 2.200999, check intersections : 8.388868
60/2851 : getting df : 3.787316, check intersections : 33.716295
80/2851 : getting df : 3.821769, check intersections : 40.810629
100/2851 : getting df : 2.323462, check intersections : 11.104965
120/2851 : getting df : 4.975245, check intersections : 6.789270
140/2851 : getting df : 0.017891, check intersections : 0.009356
160/2851 : getting df : 1.871022, check intersections : 14.747087
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is outside
component is ou

In [14]:
len(CC_to_contours),len(CC)

(47902, 47956)

In [18]:
for elem in outside_osm_nodes_data.values():
    pt=elem['geometry']
    print(pt.y,pt.x)

41.5301905 9.2728831
41.5305635 9.2731909
41.4775328 9.0748836
41.4499807 9.0937934
41.587229 9.2913172
41.4600246 9.052059
41.4666776 9.0592359
41.4688175 9.0620369
41.4701004 9.073313
41.6810498 9.3746155
41.4668663 9.0712934
41.4697894 9.0714213
41.3766895 9.2219834
41.4017307 9.2457815
41.6214817 9.3378132
41.474989 9.0714937
42.1282959 9.5167255
42.3399849 9.541435
42.3944158 9.5352665
42.5132181 9.532137
42.2166717 9.5540146
42.7110945 9.4550221
42.709843 9.4550448
42.7098926 9.4550557
42.7098778 9.4550509
42.9480988 9.3563024
42.8118385 9.4898728
42.7114388 9.4549472
42.6412356 9.4516211
42.9540119 9.3468103
42.6804785 9.2831791
42.7095239 9.4557542
42.7424684 9.462366
42.6942027 9.4541162
42.7067666 9.4544419
42.70873 9.4545467
41.6772597 8.8986156
41.5215998 8.8602052
41.541995 8.8512057
41.4783947 9.0140419
41.5484707 8.8327691
41.7071524 8.7947009
41.7070999 8.7946934
41.6771324 8.9061865
41.7073942 8.7953865
41.9179995 8.7415315
41.9187432 8.740309
41.9184306 8.7408173
41.9

In [15]:
t1=time.time()
inside_osm_nodes_data,outside_osm_nodes_data={},{}
for k,cc in enumerate(CC):
    if k in CC_to_contours:
        for osm_node in cc:
            inside_osm_nodes_data[osm_node]={'contour_line_id':CC_to_contours[k],'geometry':Point(G_osm.nodes()[osm_node]['lon'],G_osm.nodes()[osm_node]['lat'])}
    else:
        for osm_node in cc:
            outside_osm_nodes_data[osm_node]={'geometry':Point(G_osm.nodes()[osm_node]['lon'],G_osm.nodes()[osm_node]['lat'])}
        
t2=time.time()
print(t2-t1)

0.9615380764007568


In [16]:
ADD_OSM_DATA=True
if ADD_OSM_DATA:
    
    t1=time.time()
    string_list=',\n'.join(['(%i,%i,ST_GeomFromText(\'%s\'))'%(osm_id,d['contour_line_id'],d['geometry'].wkt) for osm_id,d in inside_osm_nodes_data.items()])
    cmd="INSERT INTO %s (`osm_id`,`contour_line_id`,`geometry`) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)

    string_list=',\n'.join(['(%i,ST_GeomFromText(\'%s\'))'%(osm_id,d['geometry'].wkt) for osm_id,d in outside_osm_nodes_data.items()])
    cmd="INSERT INTO %s (`osm_id`,`geometry`) VALUES %s;"%(osm_nodes_table_name,string_list)
    cursor.execute(cmd)
    
    intersection=intersection.to_crs('epsg:4326')
    string_list=',\n'.join(['(%i,%i,%i,%i,%f,%f,ST_GeomFromText(\'%s\'))'%(row['id'],row['edge'][0],row['edge'][1],row['edge'][2],row['edge_coordinate'],row['elevation'],row['geometry'].wkt)  for _,row in intersection.iterrows()])
    cmd="INSERT INTO %s (`contour_line_id`,`osm_begin`,`osm_end`,`osm_key`,`edge_coordinate`,`elevation`,`geometry`) VALUES %s;"%(osm_intersections_table_name,string_list)
    cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)

48.428051233291626
